# Update Contact References from CSV

This notebook will:
1. Connect to the RDS PostgreSQL database
2. Add a `reference` column to the `contact` table (if not exists)
3. Read the CSV file with email and reference data
4. Match emails and update the reference column

## Step 1: Install Required Packages
Run this if you don't have these packages installed

In [ ]:
# Uncomment and run if needed
# !pip install psycopg2-binary pandas python-dotenv

## Step 2: Import Libraries

In [ ]:
import psycopg2
import pandas as pd
import os
from dotenv import load_dotenv

print("Libraries imported successfully!")

## Step 3: Load Environment Variables

In [ ]:
# Load .env file
load_dotenv()

# Database connection parameters
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')

print(f"Database: {DB_NAME}@{DB_HOST}")

## Step 4: Connect to Database

In [ ]:
# Create database connection
try:
    conn = psycopg2.connect(
        host=DB_HOST,
        port=DB_PORT,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        sslmode='require'
    )
    cursor = conn.cursor()
    print("✅ Successfully connected to the database!")
except Exception as e:
    print(f"❌ Error connecting to database: {e}")

## Step 5: Check Current Contact Table Structure

In [ ]:
# Check if reference column exists
cursor.execute("""
    SELECT column_name 
    FROM information_schema.columns 
    WHERE table_name = 'contact' AND column_name = 'reference'
""")

reference_exists = cursor.fetchone() is not None
print(f"Reference column exists: {reference_exists}")

# Show current columns
cursor.execute("""
    SELECT column_name, data_type 
    FROM information_schema.columns 
    WHERE table_name = 'contact'
    ORDER BY ordinal_position
""")

columns = cursor.fetchall()
print("\nCurrent contact table columns:")
for col in columns:
    print(f"  - {col[0]} ({col[1]})")

## Step 6: Add Reference Column (if not exists)

In [ ]:
if not reference_exists:
    try:
        cursor.execute("""
            ALTER TABLE contact 
            ADD COLUMN reference VARCHAR(255)
        """)
        conn.commit()
        print("✅ Reference column added successfully!")
    except Exception as e:
        conn.rollback()
        print(f"❌ Error adding column: {e}")
else:
    print("ℹ️  Reference column already exists, skipping...")

## Step 7: Load CSV Data

In [ ]:
# Read the CSV file
csv_path = './public/loa uploaded  - Sheet6.csv'

try:
    # Read CSV - it has no header, columns are: reference, email
    df = pd.read_csv(csv_path, header=None, names=['reference', 'email'])
    
    # Clean the data (remove any whitespace and carriage returns)
    df['reference'] = df['reference'].astype(str).str.strip()
    df['email'] = df['email'].astype(str).str.strip().str.lower()
    
    print(f"✅ CSV loaded successfully!")
    print(f"Total records in CSV: {len(df)}")
    print("\nFirst 5 records:")
    print(df.head())
except Exception as e:
    print(f"❌ Error loading CSV: {e}")

## Step 8: Preview Contact Emails in Database

In [ ]:
# Check how many contacts we have and their emails
cursor.execute("SELECT COUNT(*) FROM contact")
total_contacts = cursor.fetchone()[0]
print(f"Total contacts in database: {total_contacts}")

# Show sample contacts
cursor.execute("SELECT id, email FROM contact LIMIT 5")
sample_contacts = cursor.fetchall()
print("\nSample contacts:")
for contact in sample_contacts:
    print(f"  ID: {contact[0]}, Email: {contact[1]}")

## Step 9: Update References (Match and Update)

In [ ]:
# Update references based on email match
updated_count = 0
not_found_count = 0
not_found_emails = []

print("Starting update process...\n")

for index, row in df.iterrows():
    reference = row['reference']
    email = row['email']
    
    try:
        # Update contact where email matches (case-insensitive)
        cursor.execute("""
            UPDATE contact 
            SET reference = %s 
            WHERE LOWER(email) = %s
        """, (reference, email))
        
        # Check if any row was updated
        if cursor.rowcount > 0:
            updated_count += cursor.rowcount
            if updated_count % 100 == 0:
                print(f"Progress: {updated_count} contacts updated...")
        else:
            not_found_count += 1
            not_found_emails.append(email)
            
    except Exception as e:
        print(f"❌ Error updating {email}: {e}")
        conn.rollback()
        continue

# Commit all changes
conn.commit()

print(f"\n✅ Update complete!")
print(f"   - Successfully updated: {updated_count} contacts")
print(f"   - Not found in database: {not_found_count} emails")

## Step 10: Verify Updates

In [ ]:
# Check how many contacts now have references
cursor.execute("""
    SELECT COUNT(*) 
    FROM contact 
    WHERE reference IS NOT NULL AND reference != ''
""")
contacts_with_ref = cursor.fetchone()[0]

print(f"Contacts with reference: {contacts_with_ref}")

# Show sample updated contacts
cursor.execute("""
    SELECT id, email, reference 
    FROM contact 
    WHERE reference IS NOT NULL 
    LIMIT 10
""")

updated_contacts = cursor.fetchall()
print("\nSample updated contacts:")
for contact in updated_contacts:
    print(f"  ID: {contact[0]}, Email: {contact[1]}, Reference: {contact[2]}")

## Step 11: Show Emails Not Found (Optional)

In [ ]:
# Show first 20 emails that were not found in the database
if not_found_emails:
    print(f"First 20 emails not found in database:")
    for email in not_found_emails[:20]:
        print(f"  - {email}")
    
    if len(not_found_emails) > 20:
        print(f"\n... and {len(not_found_emails) - 20} more")
else:
    print("All emails from CSV were found in the database!")

## Step 12: Close Database Connection

In [ ]:
# Close cursor and connection
cursor.close()
conn.close()
print("✅ Database connection closed successfully!")